In [111]:
from mysql import connector
import pandas as pd
import os

In [132]:
conn  = connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    database='zing42'
)
 
cursor = conn.cursor()

In [113]:
equityLoc = os.path.join(os.getcwd(),"tmp","equity",'EQUITY_L.csv')
df = pd.read_csv(equityLoc)
df.head()

,Unnamed: 0,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,0,20MICRONS,20 Microns Limited,EQ,06-OCT-2008,5,1,INE144J01027,5
1,1,21STCENMGM,21st Century Management Services Limited,EQ,03-MAY-1995,10,1,INE253B01015,10
2,2,3IINFOLTD,3i Infotech Limited,EQ,22-OCT-2021,10,1,INE748C01038,10
3,3,3MINDIA,3M India Limited,EQ,13-AUG-2004,10,1,INE470A01017,10
4,4,3PLAND,3P Land Holdings Limited,EQ,19-JUL-1995,2,1,INE105C01023,2


In [114]:

columns = list(df.columns)
columns[0] = "ID"
columns = [i.strip() for i in columns]

dtypes = list(df.dtypes)

for i in range(len(dtypes)):
    if dtypes[i]=="int64":
        dtypes[i]="INT"
    else:
        dtypes[i]="VARCHAR(255)"

cond = ["NOT NULL"]*len(dtypes)

df.columns=columns

print(dtypes)

['INT', 'VARCHAR(255)', 'VARCHAR(255)', 'VARCHAR(255)', 'VARCHAR(255)', 'INT', 'INT', 'VARCHAR(255)', 'INT']


In [115]:
sql="CREATE TABLE `EQUITY` ("
for i in range(len(columns)):
    if i:
        sql=sql+', '
    str = "`{}` {} {}".format(columns[i],dtypes[i],cond[i])
    sql=sql+str
sql=sql+")  ENGINE = InnoDB;"
cursor.execute(sql)

In [116]:
for ind in df.index:
    lst=[]
    
    for col in columns:
        lst.append(df[col][ind])
    
    query = """INSERT INTO `equity` (`ID`, `SYMBOL`, `NAME OF COMPANY`, `SERIES`, `DATE OF LISTING`, `PAID UP VALUE`, `MARKET LOT`, `ISIN NUMBER`, `FACE VALUE`) VALUES ("{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}");""".format(*lst)

    cursor.execute(query)
    conn.commit()

In [117]:
bhavLoc = os.path.join(os.getcwd(),"tmp","bhavcopy",'cm27DEC2022bhav.csv')
df = pd.read_csv(bhavLoc)
df.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13
0,182D020223,TB,99.00,99.00,99.00,99.00,99.00,97.15,1300,128700.0,27-DEC-2022,2,IN002022Y187,NaN
1,182D090223,TB,98.90,99.00,98.90,98.90,98.90,98.85,3200,316580.0,27-DEC-2022,5,IN002022Y195,NaN
2,182D090323,TB,99.00,99.00,99.00,99.00,99.00,97.00,600,59400.0,27-DEC-2022,1,IN002022Y245,NaN
3,182D200423,TB,94.08,94.08,94.08,94.08,94.08,96.76,100,9408.0,27-DEC-2022,1,IN002022Y302,NaN
4,20MICRONS,EQ,87.90,89.90,86.00,88.25,87.90,86.85,130074,11480907.2,27-DEC-2022,2887,INE144J01027,NaN


In [120]:

columns = list(df.columns)
columns = [i.strip() for i in columns]

dtypes = list(df.dtypes)

for i in range(len(dtypes)):
    if dtypes[i]=="int64":
        dtypes[i]="INT"
    else:
        dtypes[i]="VARCHAR(255)"

cond = ["NOT NULL"]*len(dtypes)
print(cond)
print(dtypes)
print(columns)

['NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL', 'NOT NULL']
['VARCHAR(255)', 'VARCHAR(255)', 'VARCHAR(255)', 'VARCHAR(255)', 'VARCHAR(255)', 'VARCHAR(255)', 'VARCHAR(255)', 'VARCHAR(255)', 'INT', 'VARCHAR(255)', 'VARCHAR(255)', 'INT', 'VARCHAR(255)', 'VARCHAR(255)']
['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'LAST', 'PREVCLOSE', 'TOTTRDQTY', 'TOTTRDVAL', 'TIMESTAMP', 'TOTALTRADES', 'ISIN', 'Unnamed: 13']


In [122]:
sql="CREATE TABLE `BHAVCOPY` ("
for i in range(len(columns)):
    if i:
        sql=sql+', '
    str = "`{}` {} {}".format(columns[i],dtypes[i],cond[i])
    sql=sql+str
sql=sql+")  ENGINE = InnoDB;"
cursor.execute(sql)

In [123]:
for ind in df.index:
    lst=[]
    
    for col in columns:
        lst.append(df[col][ind])

    query = """INSERT INTO `bhavcopy` (`SYMBOL`, `SERIES`, `OPEN`, `HIGH`, `LOW`, `CLOSE`, `LAST`, `PREVCLOSE`, `TOTTRDQTY`, `TOTTRDVAL`, `TIMESTAMP`, `TOTALTRADES`, `ISIN`, `Unnamed: 13`) VALUES ("{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}");""".format(*lst)

    cursor.execute(query)
    conn.commit()

In [167]:
q="""SELECT *, ((close-open)/close)*100 as gain
FROM bhavcopy
order by gain desc LIMIT 25"""
cursor.execute(q)

myresult = cursor.fetchall()
for x in myresult:
  print(x)


('NPST', 'SM', '120.0', '154.0', '120.0', '151.4', '154.0', '140.1', 75200, '9866480.0', '27-DEC-2022', 30, 'INE0FFK01017', 'nan', 20.73976221928666)
('SANDESH', 'EQ', '790.55', '964.1', '790.55', '964.1', '964.1', '803.45', 81669, '74663923.35', '27-DEC-2022', 5759, 'INE583B01015', 'nan', 18.0012446841614)
('CENTUM', 'EQ', '634.9', '757.7', '624.5', '752.6', '757.7', '631.45', 464420, '336163046.75', '27-DEC-2022', 19861, 'INE320B01020', 'nan', 15.639117725219245)
('JINDALPHOT', 'EQ', '299.95', '347.45', '294.0', '347.45', '347.45', '289.55', 98317, '33044134.45', '27-DEC-2022', 4301, 'INE796G01012', 'nan', 13.671031803137144)
('NAVA', 'EQ', '211.0', '245.65', '207.65', '242.75', '243.3', '210.05', 4357698, '1019804213.25', '27-DEC-2022', 50108, 'INE725A01022', 'nan', 13.079299691040164)
('SREIBNPNCD', 'NR', '160.0', '183.95', '160.0', '183.28', '183.95', '180.0', 295, '52322.75', '27-DEC-2022', 3, 'INE881J08607', 'nan', 12.701876909646442)
('SONAMCLOCK', 'EQ', '44.95', '53.8', '44.85